In [1]:
# from connect_four.game_play import game_play

In [2]:
import numpy as np
from connect_four import Game, GameType

In [3]:
game_init = np.array(
    [
        [2, 2, 2, 1, 0, 1, 0],
        [2, 1, 1, 1, 0, 2, 0],
        [1, 2, 2, 2, 0, 1, 0],
        [2, 1, 1, 1, 0, 2, 0],
        [1, 1, 1, 2, 0, 2, 0],
        [2, 2, 1, 2, 0, 1, 0],
    ]
)

In [4]:
# game = Game(game_state=None, game_type=GameType.MCTS_VS_RANDOM)
game = Game(game_state=None, game_type=GameType.HUMAN_VS_MCTS)

In [5]:
game.play()

SystemExit: Quitting game upon user request ...

/home/aabril/code/RL-assignments/.venv/lib/python3.12/site-packages/IPython/core/interactiveshell.py:3585: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)
